In [ ]:
import time
import h5py
import lmdb
import os
import time
from PIL import Image

In [ ]:
import os
import cv2

def load_images(directory, num_images):
    image_paths = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.png'):  # Change the file extension if necessary
                image_paths.append(os.path.join(root, file))

    loaded_images = []
    count = 0

    for image_path in image_paths:
        image = cv2.imread(image_path)
        loaded_images.append(image)
        count += 1

        if count >= num_images:
            break

    return loaded_images

# Example usage
directory_path = r'C:\Users\Miloszek\Desktop\wave'  # Replace with your directory path
num_images_to_load = 300000  # Specify the number of images to load

loaded_images = load_images(directory_path, num_images_to_load)

In [ ]:
# Display the image
cv2.imshow('Image', loaded_images[17])
cv2.waitKey(0)  # Wait for a key press to close the window
cv2.destroyAllWindows()

In [ ]:
parent_directory = r"C:\Users\Miloszek\Desktop"

#os.mkdir(parent_directory + '\disk')
#os.mkdir(parent_directory + '\hdf5')
#os.mkdir(parent_directory + '\lmdb')
#os.mkdir(parent_directory + '\_tf')
#os.mkdir(parent_directory + '\_arrow')

In [ ]:
def count_images(folder_path):
    count = 0

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            # Check if the file has an image extension
            if file.lower().endswith('.png'):
                count += 1

    return count

# Example usage:
folder_path = r"C:\Users\Miloszek\Desktop\wave"
image_count = count_images(folder_path)
print(f"Total images found: {image_count}")

In [ ]:
def remove(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)

        # Check if the current path is a file
        if os.path.isfile(file_path):
            # Delete the file
            os.remove(file_path)
            
remove(r"C:\Users\Miloszek\Desktop\_tf")
remove(r"C:\Users\Miloszek\Desktop\disk")
remove(r"C:\Users\Miloszek\Desktop\hdf5")
remove(r"C:\Users\Miloszek\Desktop\lmdb")

In [ ]:
import time
import numpy as np

def save_images(images):
    
    start_time = time.time()
    for i, image in enumerate(images):
        image_path = os.path.join(r"C:\Users\Miloszek\Desktop\disk", f'image_{i}.png')
        cv2.imwrite(image_path, image)
    end_time = time.time()

    elapsed_time = end_time - start_time
    return elapsed_time

def save_images_to_hdf5(images):
    
    images = np.array(images)
    
    start_time = time.time()
    with h5py.File(r"C:\Users\Miloszek\Desktop\hdf5\data.h5", 'w') as file:
        dataset = file.create_dataset('images', data=images)
    end_time = time.time()

    elapsed_time = end_time - start_time
    return elapsed_time


def save_images_to_lmdb(images):
    
    image_size_bytes = images[0].nbytes
    map_size = image_size_bytes * len(images) * 10  # Set the map size based on the images size and number of images

    env = lmdb.open(r"C:\Users\Miloszek\Desktop\lmdb\data.lmdb", map_size=map_size)

    start_time = time.time()
    with env.begin(write=True) as txn:
        for i, image in enumerate(images):
            key = f"{i:08}".encode()
            value = image.tobytes()
            txn.put(key, value)
    end_time = time.time()

    elapsed_time = end_time - start_time
    return elapsed_time



def save_images_to_tfrecord(images):
    start_time = time.time()
    num_samples = len(images)
    tf_writer = tf.io.TFRecordWriter(r"C:\Users\Miloszek\Desktop\_tf\data.tfrecord")

    for i in range(num_samples):
        image = images[i]

        feature = {
            'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image.tobytes()])),
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        tf_writer.write(example.SerializeToString())

    tf_writer.close()
    end_time = time.time()

    elapsed_time = end_time - start_time
    return elapsed_time

In [ ]:
#saving_time = save_images(loaded_images)
#print("Time taken to save {} images on disk: {:.3f} seconds".format(num_images_to_load, saving_time))
saving_time = save_images_to_hdf5(loaded_images)
print("Time taken to save {} images to hdf5: {:.3f} seconds".format(num_images_to_load, saving_time))
saving_time = save_images_to_lmdb(loaded_images)
print("Time taken to save {} images to lmdb: {:.3f} seconds".format(num_images_to_load, saving_time))
saving_time = save_images_to_tfrecord(loaded_images)
print("Time taken to save {} images to tfrecord: {:.3f} seconds".format(num_images_to_load, saving_time))

In [ ]:
def read_images_from_png():
    filepaths = r"C:\Users\Miloszek\Desktop\disk"
    images = []
    start_time = time.time()
    for filepath in filepaths:
        image = cv2.imread(filepath)
        images.append(image)
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

def read_images_from_hdf5():
    filepath = r"C:\Users\Miloszek\Desktop\hdf5\data.h5"
    images = []
    start_time = time.time()
    with h5py.File(filepath, 'r') as file:
        images = file['images']
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

def read_images_from_lmdb():
    filepath = r"C:\Users\Miloszek\Desktop\lmdb\data.lmdb"
    images = []
    start_time = time.time()
    env = lmdb.open(filepath, readonly=True)
    with env.begin() as txn:
        cursor = txn.cursor()
        for image_id in range(100):
            data = txn.get(f"{image_id:08}".encode("ascii"))
            images.append(data)
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

def parse_tfrecord_fn(example):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.decode_raw(example['image'], tf.uint8)
    return image

def read_images_from_tfrecord():
    filepath = r"C:\Users\Miloszek\Desktop\_tf\data.tfrecord"
    start_time = time.time()
    dataset = tf.data.TFRecordDataset(filepath)
    dataset = dataset.map(parse_tfrecord_fn)
    end_time = time.time()
    elapsed_time = end_time - start_time
    return elapsed_time

In [ ]:
reading_time = read_images_from_png()
print("Time taken to read {} images from disk: {:.3f} seconds".format(num_images_to_load, reading_time))
reading_time = read_images_from_hdf5()
print("Time taken to read {} images from hdf5: {:.3f} seconds".format(num_images_to_load, reading_time))
reading_time = read_images_from_lmdb()
print("Time taken to read {} images from lmdb: {:.3f} seconds".format(num_images_to_load, reading_time))
reading_time = read_images_from_tfrecord()
print("Time taken to read {} images from tfrecord: {:.3f} seconds".format(num_images_to_load, reading_time))

In [ ]:
def get_folder_size(folder_path):
    total_size = 0

    for path, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(path, file)
            total_size += os.path.getsize(file_path)

    return total_size

size = get_folder_size(r"C:\Users\Miloszek\Desktop\disk")
print(f"DS size: {(size / (1024 * 1024 * 1024)):.2f} GB")
size = get_folder_size(r"C:\Users\Miloszek\Desktop\hdf5")
print(f"DS size: {(size / (1024 * 1024 * 1024)):.2f} GB")
size = get_folder_size(r"C:\Users\Miloszek\Desktop\lmdb")
print(f"DS size: {(size / (1024 * 1024 * 1024)):.2f} GB")
size = get_folder_size(r"C:\Users\Miloszek\Desktop\_tf")
print(f"DS size: {(size / (1024 * 1024 * 1024)):.2f} GB")

In [ ]:
import tensorflow as tf

train_path = r"C:\Users\Miloszek\Desktop\wave\train"
valid_path = r"C:\Users\Miloszek\Desktop\wave\valid"

# Set the image dimensions
image_size = (67, 193)  # Adjust to your desired image size

# Set batch size
batch_size = 32  # Adjust batch size as needed

# Load the image dataset from the directory without splitting
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_path,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=27,
    label_mode='binary',
    validation_split=0.0  # Set to 0.0 to avoid splitting the dataset
)

valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    valid_path,
    image_size=image_size,
    batch_size=batch_size,
    shuffle=True,
    seed=27,
    label_mode='binary',
    validation_split=0.0  # Set to 0.0 to avoid splitting the dataset
)

In [ ]:
import efficientnet.keras as efn
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Initialize EfficientNetB0 model
ef = efn.EfficientNetB0(input_shape=(67, 193, 3), weights='imagenet', include_top=False)

# Add your own classification layers on top
x = ef.output
x = GlobalAveragePooling2D()(x)  # Global average pooling to reduce spatial dimensions
x = Dense(128, activation='relu')(x)  # Add a fully connected layer with 128 units
x = Dense(1, activation='sigmoid')(x)  # Output layer with sigmoid activation for binary classification

# Create the final model
model = Model(inputs=ef.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
#model.summary()

In [ ]:
history = model.fit(train_ds, epochs=3, validation_data=valid_ds)
loss, accuracy = model.evaluate(test_ds)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

In [ ]:
#model.save_weights('/content/drive/MyDrive/Colab'+' '+'Notebooks/my_model_weights.h5')
#model.load_weights('/content/drive/MyDrive/Colab'+' '+'Notebooks/my_model_weights.h5')

In [ ]:
img = Image.open('/content/drive/MyDrive/Colab'+' '+'Notebooks/224/Clean/Imgclean_765_0.png')

# Preprocess the image
img_array = np.array(img)
img_array = np.expand_dims(img_array, axis=0)

# Make a prediction with the model
print(model.predict(img_array))
prediction = np.argmax(model.predict(img_array))
print('Dusty' if prediction == 0 else 'Clean')